## Ejemplo 1: Algoritmo Genético Simple

Resuelve el problema de la mochila usando evolución.

In [ ]:
import random

def algoritmo_genetico_mochila(pesos, valores, capacidad, 
                               tam_poblacion=50, generaciones=100, 
                               tasa_mutacion=0.01, tasa_cruce=0.8):
    """
    Algoritmo Genético para la mochila.
    Individuos: listas binarias [0,1,1,0,...]
    """
    n = len(pesos)
    
    # Crear individuo aleatorio
    def crear_individuo():
        return [random.randint(0, 1) for _ in range(n)]
    
    # Fitness
    def fitness(individuo):
        peso_total = sum(p * g for p, g in zip(pesos, individuo))
        valor_total = sum(v * g for v, g in zip(valores, individuo))
        if peso_total > capacidad:
            return 0  # Penalizar
        return valor_total
    
    # Selección por torneo
    def seleccion_torneo(poblacion, k=3):
        torneo = random.sample(poblacion, k)
        return max(torneo, key=fitness)
    
    # Cruce de un punto
    def cruce(padre1, padre2):
        if random.random() > tasa_cruce:
            return padre1[:], padre2[:]
        punto = random.randint(1, n - 1)
        hijo1 = padre1[:punto] + padre2[punto:]
        hijo2 = padre2[:punto] + padre1[punto:]
        return hijo1, hijo2
    
    # Mutación
    def mutar(individuo):
        for i in range(len(individuo)):
            if random.random() < tasa_mutacion:
                individuo[i] = 1 - individuo[i]
        return individuo
    
    # Inicializar población
    poblacion = [crear_individuo() for _ in range(tam_poblacion)]
    mejor_historico = None
    mejor_fitness = 0
    historial = []
    
    # Evolución
    for gen in range(generaciones):
        # Evaluar
        fitness_scores = [(ind, fitness(ind)) for ind in poblacion]
        fitness_scores.sort(key=lambda x: x[1], reverse=True)
        
        # Actualizar mejor
        if fitness_scores[0][1] > mejor_fitness:
            mejor_fitness = fitness_scores[0][1]
            mejor_historico = fitness_scores[0][0][:]
        
        historial.append(mejor_fitness)
        
        # Nueva generación
        nueva_poblacion = []
        
        # Elitismo: mantener mejores 10%
        elite_size = tam_poblacion // 10
        nueva_poblacion.extend([ind for ind, _ in fitness_scores[:elite_size]])
        
        # Generar resto
        while len(nueva_poblacion) < tam_poblacion:
            padre1 = seleccion_torneo(poblacion)
            padre2 = seleccion_torneo(poblacion)
            hijo1, hijo2 = cruce(padre1, padre2)
            hijo1 = mutar(hijo1)
            hijo2 = mutar(hijo2)
            nueva_poblacion.extend([hijo1, hijo2])
        
        poblacion = nueva_poblacion[:tam_poblacion]
    
    return mejor_historico, mejor_fitness, historial

# Prueba
pesos = [2, 3, 4, 5, 6, 7]
valores = [3, 4, 5, 8, 10, 12]
capacidad = 15

print("Algoritmo Genético - Problema de la Mochila")
print("="*60)
print(f"Capacidad: {capacidad}\n")
print("Items:")
for i in range(len(pesos)):
    print(f"  Item {i}: peso={pesos[i]}, valor={valores[i]}")

print(f"\n{'='*60}\n")

solucion, valor, historial = algoritmo_genetico_mochila(
    pesos, valores, capacidad, generaciones=50
)

print(f"Mejor solución: {solucion}")
print(f"Valor: {valor}")
print(f"\nItems seleccionados:")
peso_total = 0
for i, sel in enumerate(solucion):
    if sel:
        print(f"  Item {i}: peso={pesos[i]}, valor={valores[i]}")
        peso_total += pesos[i]
print(f"\nPeso total: {peso_total}/{capacidad}")

# Mostrar evolución
print("\nEvolución del fitness (cada 10 generaciones):")
for i in range(0, len(historial), 10):
    print(f"  Gen {i:3d}: {historial[i]}")

## Ejemplo 2: Recocido Simulado (Simulated Annealing)

Resuelve TSP usando enfriamiento simulado.

In [ ]:
import random
import math

def recocido_simulado_tsp(distancias, temp_inicial=100, temp_final=0.1, 
                          alpha=0.95, iteraciones_por_temp=100):
    """
    Recocido Simulado para TSP.
    Acepta soluciones peores con probabilidad decreciente.
    """
    n = len(distancias)
    
    def costo_tour(tour):
        costo = 0
        for i in range(len(tour)):
            costo += distancias[tour[i]][tour[(i + 1) % len(tour)]]
        return costo
    
    def vecino_2opt(tour):
        """Genera vecino intercambiando segmento."""
        nuevo = tour[:]
        i, j = sorted(random.sample(range(n), 2))
        nuevo[i:j+1] = reversed(nuevo[i:j+1])
        return nuevo
    
    # Solución inicial aleatoria
    tour_actual = list(range(n))
    random.shuffle(tour_actual)
    costo_actual = costo_tour(tour_actual)
    
    mejor_tour = tour_actual[:]
    mejor_costo = costo_actual
    
    temperatura = temp_inicial
    historial = []
    
    while temperatura > temp_final:
        for _ in range(iteraciones_por_temp):
            # Generar vecino
            tour_vecino = vecino_2opt(tour_actual)
            costo_vecino = costo_tour(tour_vecino)
            
            delta = costo_vecino - costo_actual
            
            # Aceptar o rechazar
            if delta < 0:
                tour_actual = tour_vecino
                costo_actual = costo_vecino
                
                if costo_actual < mejor_costo:
                    mejor_tour = tour_actual[:]
                    mejor_costo = costo_actual
            else:
                probabilidad = math.exp(-delta / temperatura)
                if random.random() < probabilidad:
                    tour_actual = tour_vecino
                    costo_actual = costo_vecino
        
        historial.append((temperatura, mejor_costo))
        temperatura *= alpha
    
    return mejor_tour, mejor_costo, historial

# Prueba
distancias = [
    [0, 10, 15, 20],
    [10, 0, 35, 25],
    [15, 35, 0, 30],
    [20, 25, 30, 0]
]

print("Recocido Simulado - TSP")
print("="*60)
print("Matriz de distancias:")
for fila in distancias:
    print(" ".join(f"{d:3d}" for d in fila))

print(f"\n{'='*60}\n")

tour, costo, historial = recocido_simulado_tsp(distancias)

print(f"Mejor tour: {tour}")
print(f"Costo: {costo}")

print("\nEnfriamiento (cada 5 pasos):")
for i in range(0, len(historial), 5):
    temp, cost = historial[i]
    print(f"  T={temp:6.2f}: costo={cost}")

## Ejemplo 3: Hill Climbing (Búsqueda Local)

Búsqueda local simple que siempre mejora.

In [ ]:
import random

def hill_climbing(func_objetivo, dim, rango, max_iter=1000):
    """
    Hill Climbing simple para optimización.
    Siempre se mueve a la mejor solución vecina.
    """
    # Solución inicial aleatoria
    actual = [random.uniform(rango[0], rango[1]) for _ in range(dim)]
    valor_actual = func_objetivo(actual)
    
    historial = [valor_actual]
    mejoras = 0
    
    for iteracion in range(max_iter):
        # Generar vecinos (pequeñas perturbaciones)
        mejor_vecino = None
        mejor_valor = valor_actual
        
        for _ in range(10):  # Probar 10 vecinos
            vecino = actual[:]
            # Perturbar dimensión aleatoria
            idx = random.randint(0, dim - 1)
            vecino[idx] += random.uniform(-0.5, 0.5)
            vecino[idx] = max(rango[0], min(rango[1], vecino[idx]))
            
            valor_vecino = func_objetivo(vecino)
            if valor_vecino > mejor_valor:
                mejor_vecino = vecino
                mejor_valor = valor_vecino
        
        # Si encontramos mejora, moverse
        if mejor_vecino is not None:
            actual = mejor_vecino
            valor_actual = mejor_valor
            mejoras += 1
        else:
            # Óptimo local alcanzado
            break
        
        historial.append(valor_actual)
    
    return actual, valor_actual, historial, mejoras

# Función de prueba: maximizar -(x^2 + y^2) (máximo en (0,0))
def funcion_prueba(x):
    return -(x[0]**2 + x[1]**2)

print("Hill Climbing")
print("="*60)
print("Maximizar: f(x,y) = -(x² + y²)")
print("Máximo en (0, 0) con valor 0\n")

# Ejecutar varias veces
print("10 ejecuciones:")
resultados = []
for i in range(10):
    sol, val, hist, mej = hill_climbing(funcion_prueba, 2, (-10, 10))
    resultados.append((sol, val))
    print(f"  {i+1}. Solución: ({sol[0]:.4f}, {sol[1]:.4f}), Valor: {val:.4f}")

mejor_sol, mejor_val = max(resultados, key=lambda x: x[1])
print(f"\nMejor encontrado: ({mejor_sol[0]:.4f}, {mejor_sol[1]:.4f})")
print(f"Valor: {mejor_val:.4f}")
print(f"Error desde óptimo: {abs(mejor_val):.4f}")

## 🎯 Ejercicio Práctico

Implementa una búsqueda tabú simple.

In [ ]:
from collections import deque

def busqueda_tabu_simple(func_objetivo, dim, rango, 
                         tamano_tabu=10, max_iter=100):
    """
    Búsqueda Tabú simple.
    Mantiene lista de movimientos prohibidos.
    """
    # Solución inicial
    actual = [random.uniform(rango[0], rango[1]) for _ in range(dim)]
    valor_actual = func_objetivo(actual)
    
    mejor = actual[:]
    mejor_valor = valor_actual
    
    lista_tabu = deque(maxlen=tamano_tabu)
    historial = [mejor_valor]
    
    for _ in range(max_iter):
        # Generar vecinos no tabú
        mejor_vecino = None
        mejor_valor_vecino = float('-inf')
        mejor_mov = None
        
        for _ in range(20):  # Probar 20 vecinos
            vecino = actual[:]
            idx = random.randint(0, dim - 1)
            delta = random.uniform(-1, 1)
            vecino[idx] += delta
            vecino[idx] = max(rango[0], min(rango[1], vecino[idx]))
            
            movimiento = (idx, delta)
            
            # Verificar si no es tabú o cumple criterio de aspiración
            valor_vecino = func_objetivo(vecino)
            if (movimiento not in lista_tabu or valor_vecino > mejor_valor):
                if valor_vecino > mejor_valor_vecino:
                    mejor_vecino = vecino
                    mejor_valor_vecino = valor_vecino
                    mejor_mov = movimiento
        
        if mejor_vecino is None:
            break
        
        # Moverse
        actual = mejor_vecino
        valor_actual = mejor_valor_vecino
        lista_tabu.append(mejor_mov)
        
        # Actualizar mejor global
        if valor_actual > mejor_valor:
            mejor = actual[:]
            mejor_valor = valor_actual
        
        historial.append(mejor_valor)
    
    return mejor, mejor_valor, historial

# Prueba
print("Búsqueda Tabú")
print("="*60)

sol, val, hist = busqueda_tabu_simple(funcion_prueba, 2, (-10, 10))
print(f"Solución: ({sol[0]:.4f}, {sol[1]:.4f})")
print(f"Valor: {val:.4f}")
print(f"\nMejora por iteración (primeras 10):")
for i, v in enumerate(hist[:10]):
    print(f"  Iteración {i}: {v:.4f}")

## 🎓 Conclusiones del Curso

¡Felicidades! Has completado el curso de Estrategias Algorítmicas.

### Resumen de lo Aprendido:

1. **Fuerza Bruta** - Base conceptual, exploración exhaustiva
2. **Dividir y Conquistar** - Descomponer y combinar
3. **Programación Dinámica** - Memoización y optimización
4. **Algoritmos Voraces** - Decisiones locales óptimas
5. **Backtracking** - Exploración con retroceso
6. **Ramificación y Acotación** - Poda con cotas
7. **Probabilísticos** - Poder de la aleatoriedad
8. **Heurísticos** - Soluciones prácticas aproximadas

### Próximos Pasos:

- 💻 Practica en plataformas como LeetCode, Codeforces
- 📚 Profundiza en algoritmos específicos de tu área
- 🏆 Participa en competencias de programación
- 🔬 Aplica estos conceptos a problemas reales

**¡Éxito en tu viaje algorítmico! 🚀**